In [7]:
import numpy as np
import sys
sys.path.append('..')
from custom_lib import quantum_circuit_ctx_generator, utils, fixed_point_handler
from qiskit import QuantumCircuit
import qiskit
from qimax import converter, constant

In [30]:
def qft(n):
    """Creates a QFT circuit on n qubits using only 1-qubit gates and CX."""
    qc = QuantumCircuit(n)
    
    for j in range(n):
        qc.h(j)  # Hadamard gate
        for k in range(j+1, n):
            angle = np.pi / (2 ** (k - j))  # Correct phase shift
            qc.cx(k, j)
            qc.rz(angle / 2, j)  # Phase rotation on target qubit
            qc.cx(k, j)
            qc.rz(-angle / 2, j)  # Undo unwanted phase shift
    # Swap qubits at the end to match QFT output order
    for i in range(n // 2):

        qc.cx(j, n-j-1)
        qc.cx(n-j-1, j)
        qc.cx(j, n-j-1)
    return qc


def generate_QFT_quantum_circuit_data_with_state_res_local(num_qubits=3, output_file_path='output.txt', output_original_state_path='output_original_state.txt', output_state_path='output_state.txt'):
    qc_trans = qft(num_qubits)
    # print(qiskit.quantum_info.Statevector.from_instruction(qc).data)
        
    # qc_trans = qiskit.transpile(qc, basis_gates=['h', 's', 'cx', 'rx', 'ry', 'rz'])
    qc_trans.draw(output='mpl')
    # print(qiskit.quantum_info.Statevector.from_instruction(qc_trans).data)

    
    print(qiskit.quantum_info.Statevector.from_instruction(qc_trans).data)
    qc_trans.draw("mpl")

    # Convert to readable files
    texts = []
    texts.append(len(qc_trans.data))
    for gate in qc_trans.data:
        name = gate.name.upper()
        params = None
        wires = converter.get_wires_of_gate(gate)
        if name == 'CX':
            type = 2
        else:
            type = 1
            gate_entries = constant.constant_gate[name]
            if name in ['RX', 'RY', 'RZ']:
                params = gate.params[0]
                gate_entries = gate_entries(params)
        texts.append(type)
        if type == 2:
            texts.append("{" + str(wires[0]) + ", "+ str(wires[1]) + "}")
        else:
            texts.append(wires[0])
            texts.append(gate_entries[0][0])
            texts.append(gate_entries[0][1])
            texts.append(gate_entries[1][0])
            texts.append(gate_entries[1][1])

    with open(output_file_path, 'w') as f:
        for item in texts:
            f.write("%s\n" % item)

    # phase = np.e**(1j*qc_trans.global_phase)
    # if qc_trans.global_phase > np.pi:
    #     phase *= -1

    # # FROM ORIGINAL CIRCUITS
    # qc_qiskit_original = qiskit.quantum_info.Statevector.from_instruction(qc_trans).data

    # THIS IS THE EXPECTED OUTPUT
    qc_qiskit_processed = qiskit.quantum_info.Statevector.from_instruction(qc_trans).data

    # res_state_0 = np.array(qc_qiskit_original)
    # np.savetxt(output_original_state_path, res_state_0)

    res_state_1 = np.array(qc_qiskit_processed)
    np.savetxt(output_state_path, res_state_1)


def quantum_circuit_computation_qft_local(n_qubit=3, ctx_link='output.txt'):
    gate_ctx_data = utils.read_file(ctx_link)

    Dense_matrix = np.array([[0, 0], [0, 0]], dtype=complex)

    state_vector = np.zeros(2**n_qubit, dtype=complex)
    state_vector[0] = 1

    bit_width = 32
    fraction_width = 30

    sub_idx = 0
    gate_type = 1 # 0: Sprase, 1: Dense, 2: CX
    num_of_gates = 0
    gate_count = 0
    for index, line in enumerate(gate_ctx_data):
        if index == 0: # Number of gates
            num_of_gates = int(line.strip())
            print(f"=> Number of gates: {num_of_gates}")
        else:
            if(sub_idx == 0): # Process gate type
                print(f"==================== Gate {gate_count} ====================")
                gate_count = gate_count + 1
                gate_type = int(line.strip())
                sub_idx = sub_idx + 1
            elif(sub_idx == 1):
                if(gate_type == 1): # Process dense gate position
                    pos = n_qubit-1-int(line.strip())
                    # pos = int(line.strip())
                    print(f"====> Dense gate | Pos: {pos}")
                    sub_idx = sub_idx + 1
                elif(gate_type == 2): # Process CX gate position
                    sub_idx = 0
                    ctrl_pos, tgt_pos = line.strip("{}").split(", ")
                    ctrl_pos = n_qubit-1-int(ctrl_pos)
                    tgt_pos = n_qubit-1-int(tgt_pos)
                    # ctrl_pos = int(ctrl_pos)
                    # tgt_pos = int(tgt_pos)

                    print(f"====> CX gate | Ctrl Pos: {ctrl_pos} | Tgt Pos: {tgt_pos}")

                    cx_matrix = utils.cx_matrix_normal(n_qubit, ctrl_pos, tgt_pos)
                    # cx_matrix = utils.cx_matrix_formula(n_qubit, ctrl_pos, tgt_pos)
                    state_vector = np.dot(cx_matrix, state_vector)

                    print(f"State vector after CX gate:")
                    for i in range(2**n_qubit):
                        real_part = state_vector[i].real
                        imaginary_part = state_vector[i].imag

                        real_part_binary = fixed_point_handler.convert_fixedpoint_to_binary(number=real_part, bit_width=bit_width, dot_position=fraction_width)
                        imaginary_part_binary = fixed_point_handler.convert_fixedpoint_to_binary(number=imaginary_part, bit_width=bit_width, dot_position=fraction_width)

                        real_part_hex = fixed_point_handler.binary_to_hex(binary_str=real_part_binary, bit_width=bit_width)
                        imaginary_part_hex = fixed_point_handler.binary_to_hex(binary_str=imaginary_part_binary, bit_width=bit_width)

                        # if(i < 16):
                        print(f"Complex number: {state_vector[i]} | Hex form: {real_part_hex[2:]+imaginary_part_hex[2:]}")
            else: # Process dense gate entries
                if(gate_type):
                    if(line[0] == '('):
                        line = line.strip("()")
                        
                    line = line.strip("")
                    compl_num = complex(line)

                    if(sub_idx == 2):
                        Dense_matrix[0][0] = compl_num
                    elif(sub_idx == 3):
                        Dense_matrix[0][1] = compl_num
                    elif(sub_idx == 4):
                        Dense_matrix[1][0] = compl_num
                    elif(sub_idx == 5):
                        Dense_matrix[1][1] = compl_num

                        full_dense_matrix = utils.create_full_gate_matrix(gate_matrix=Dense_matrix, gate_pos=pos, n_qubit=n_qubit)
                        state_vector = np.dot(full_dense_matrix, state_vector)
                        print(f"State vector after Dense gate:")
                        for i in range(2**n_qubit):
                            real_part = state_vector[i].real
                            imaginary_part = state_vector[i].imag

                            real_part_binary = fixed_point_handler.convert_fixedpoint_to_binary(number=real_part, bit_width=bit_width, dot_position=fraction_width)
                            imaginary_part_binary = fixed_point_handler.convert_fixedpoint_to_binary(number=imaginary_part, bit_width=bit_width, dot_position=fraction_width)

                            real_part_hex = fixed_point_handler.binary_to_hex(binary_str=real_part_binary, bit_width=bit_width)
                            imaginary_part_hex = fixed_point_handler.binary_to_hex(binary_str=imaginary_part_binary, bit_width=bit_width)

                            # if(i > 255 and i < 264):
                            # if(i < 8):
                            #     print(f"Complex number: {state_vector[i]} | Hex form: {real_part_hex[2:]+imaginary_part_hex[2:]}")
                            print(f"Complex number: {state_vector[i]} | Hex form: {real_part_hex[2:]+imaginary_part_hex[2:]}")

                    if(sub_idx < 5):
                        sub_idx = sub_idx + 1
                    else:
                        sub_idx = 0

In [31]:

min_qbit_num = 3
max_qbit_num = 3
qubit_num_idx_range = np.arange(min_qbit_num, max_qbit_num+1, 1)

utils.create_folder('./test/')
for qubit_num_idx in qubit_num_idx_range:
    saved_folder = './test/quantum_circuit_data_' + str(qubit_num_idx) + '_qubits/'
    utils.create_folder(saved_folder)
    raw_file_path = saved_folder + 'output_QFT_' + str(qubit_num_idx) + '_qubits' + '.txt'
    processed_file_path = saved_folder + 'output_hex_QFT_' + str(qubit_num_idx) + '_qubits' + '.txt'
    state_file_original_path = saved_folder + 'output_original_state_QFT_' + str(qubit_num_idx) + '_qubits' + '.txt'
    state_file_path = saved_folder + 'output_state_QFT_' + str(qubit_num_idx) + '_qubits' + '.txt'
    generate_QFT_quantum_circuit_data_with_state_res_local(num_qubits=qubit_num_idx, 
                                                                                   output_file_path=raw_file_path, 
                                                                                   output_original_state_path=state_file_original_path, 
                                                                                   output_state_path=state_file_path)
    quantum_circuit_ctx_generator.generate_quantum_circuit_ctx(n_qubit=qubit_num_idx, bit_width=32, fraction_width=30, raw_file_path=raw_file_path, processed_file_path=processed_file_path)

Directory './test/' already exists.
Directory './test/quantum_circuit_data_3_qubits/' already exists.
[0.35355339+1.96261557e-17j 0.35355339+1.96261557e-17j
 0.35355339+1.96261557e-17j 0.35355339+1.96261557e-17j
 0.35355339-1.96261557e-17j 0.35355339-1.96261557e-17j
 0.35355339-1.96261557e-17j 0.35355339-1.96261557e-17j]


In [32]:
n_qubit = 3
ctx_link = './test/quantum_circuit_data_' + str(n_qubit) + '_qubits/' + 'output_QFT_' + str(n_qubit) + '_qubits.txt'

quantum_circuit_computation_qft_local(n_qubit=n_qubit, ctx_link=ctx_link)

=> Number of gates: 18
==================== Gate 0 ====================
====> Dense gate | Pos: 2
State vector after Dense gate:
Complex number: (0.7071067811865475+0j) | Hex form: 2d413ccc00000000
Complex number: (0.7071067811865475+0j) | Hex form: 2d413ccc00000000
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
==================== Gate 1 ====================
====> CX gate | Ctrl Pos: 1 | Tgt Pos: 2
State vector after CX gate:
Complex number: (0.7071067811865475+0j) | Hex form: 2d413ccc00000000
Complex number: (0.7071067811865475+0j) | Hex form: 2d413ccc00000000
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 00000000000000

In [33]:
import numpy as np
import sys
sys.path.append('..')
from custom_lib import quantum_circuit_ctx_generator, utils

min_qbit_num = 3
max_qbit_num = 17
min_quantum_circuit_idx = 1
max_quantum_circuit_idx = 19

quantum_circuit_idx_range = np.arange(min_quantum_circuit_idx, max_quantum_circuit_idx+1, 1)
qubit_num_idx_range = np.arange(min_qbit_num, max_qbit_num+1, 1)

utils.create_folder('../hardware/gate_ctx_for_sim/QFT/')
for qubit_num_idx in qubit_num_idx_range:
    saved_folder = '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_' + str(qubit_num_idx) + '_qubits/'
    utils.create_folder(saved_folder)
    raw_file_path = saved_folder + 'output_QFT_' + str(qubit_num_idx) + '_qubits' + '.txt'
    processed_file_path = saved_folder + 'output_hex_QFT_' + str(qubit_num_idx) + '_qubits' + '.txt'
    state_file_original_path = saved_folder + 'output_original_state_QFT_' + str(qubit_num_idx) + '_qubits' + '.txt'
    state_file_path = saved_folder + 'output_state_QFT_' + str(qubit_num_idx) + '_qubits' + '.txt'
    # quantum_circuit_ctx_generator.generate_QFT_quantum_circuit_data_with_state_res(num_qubits=qubit_num_idx, 
    #                                                                                output_file_path=raw_file_path, 
    #                                                                                output_original_state_path=state_file_original_path, 
    #                                                                                output_state_path=state_file_path)
    quantum_circuit_ctx_generator.generate_quantum_circuit_ctx(n_qubit=qubit_num_idx, bit_width=32, fraction_width=30, raw_file_path=raw_file_path, processed_file_path=processed_file_path)

Directory '../hardware/gate_ctx_for_sim/QFT/' already exists.
Directory '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_3_qubits/' already exists.
Directory '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_4_qubits/' already exists.
Directory '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_5_qubits/' already exists.
Directory '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_6_qubits/' already exists.
Directory '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_7_qubits/' already exists.
Directory '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_8_qubits/' already exists.
Directory '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_9_qubits/' already exists.
Directory '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_10_qubits/' already exists.
Directory '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_11_qubits/' already exists.
Directory '../hardware/gate_ctx_for_sim/QFT/quantum_circuit_data_12_qubits/' already exists.
Directory '../h